<a href="https://colab.research.google.com/github/seopp/Tobigs19/blob/main/Week1_Framework_%EC%9E%84%EC%8A%B9%EC%84%AD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week1_Library 과제

### Q1. Library 와 Framework 의 차이 간단하게 서술하시오. (100자 내외)

Framework는 원하는 기능 구현에 집중하여 개발할 수 있도록 일정한 형태와 필요한 기능을 갖추고 있는 골격, 뼈대를 의미한다.  
Library는 소프트웨어를 개발할 때 컴퓨터 프로그램이 사용하는 비휘발성 자원의 모임. 즉 특정 기능을 모와둔 코드, 함수들의 집합이며 코드 작성 시 활용 가능한 도구들을 의미한다.

프레임워크와 라이브러리의 차이점은 "제어 흐름"의 권한이 어디에 있는가이다.

라이브러리를 사용할 때 사용자는 애플리케이션 코드의 흐름을 직접 제어해야 한다.
개발 시 필요한 기능이 있을 경우 능동적으로 라이브러리를 호출하여 사용하거나 기존에 구성된 함수나 코드를 가져다 써야 한다.

반면 프레임워크는 애플리케이션의 코드가 프레임워크에 의해 사용된다.
애플리케이션 코드는 프레임워크가 짜 놓은 틀에서 수동적으로 동작하기 때문에 제어의 흐름은 프레임워크가 가지고 있고 사용자가 그 안에 필요한 코드를 작성하게 된다.



### Q2. 딥러닝과 머신러닝의 관계 및 특징, 차이 간단하게 서술하시오. (200자 내외)

인공지능 > 머신러닝 > 딥러닝 순으로 포괄적인 관계를 가지고 있다. 

딥러닝은 머신러닝과 마찬가지로 인공지능의 하위 개념이며, 인공신경망에서 발전한 형태이다. 인공신경망(Artificial Neural Network)은 뇌의 뉴런과 유사한 정보 입출력 계층을 활용한 것으로, 블랙박스 형태로 데이터를 입력하면 자동으로 복잡한 수학식 모델링이 되는 기법이다. 딥러닝은 이러한 복잡한 인공신경망을 사용한 알고리즘을 통해 데이터를 학습한다.

머신러닝은 알고리즘이 부정확한 예측을 반환하면 엔지니어가 개입하여 조정해야 한다. 그러나 딥러닝은 알고리즘 자체 신경망을 통해 예측 정확성 여부를 스스로 판단한다. 또한, 딥러닝은 추상적인 정보를 인식하는 능력이 뛰어나기 때문에 머신러닝과 달리 개와 고양이를 식별할 수 있다. 머신러닝은 엔지니어가 미리 각 데이터가 개인지 고양이인지를 정의 내려야 하는 반면, 딥러닝은 개의 데이터와 개가 아닌 데이터들이 주어지면 자동으로 개인지 아닌지를 군집화하고 분류한다.



### Q3. 아래의 코드에 주석 달기.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transfroms

device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(45)
if device == 'cuda':
    torch.cuda.manual_seed_all(45)
print(device + " is available")

In [ ]:
learning_rate = 0.001
batch_size = 100
num_classes = 10
epochs = 5

In [ ]:
# MNIST dataset을 train과 test로 나누어 불러온다.
train_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = True,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor() 
    ])
)

test_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = False,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor()
    ])
)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)

examples = enumerate(train_set)
batch_idx, (example_data, example_targets) = next(examples)
example_data.shape

In [ ]:
# Convnet의 class를 만든다. 
# Pytorch의 nn.Module을 상속받아 __init__()과 forward() 함수를 오버라이딩 해준다.
class ConvNet(nn.Module):
  def __init__(self): 
        super(ConvNet, self).__init__() 
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) 
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5) 
        self.drop2D = nn.Dropout2d(p=0.25, inplace=False) 
        self.mp = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(320,100) 
        self.fc2 = nn.Linear(100,10) 


  def forward(self, x):
        x = F.relu(self.mp(self.conv1(x))) 
        x = F.relu(self.mp(self.conv2(x))) 
        x = self.drop2D(x) 
        x = x.view(x.size(0), -1) 
        x = self.fc1(x)
        x = self.fc2(x)
        return F.log_softmax(x)

In [ ]:
# 모델의 손실함수와 최적화함수를 정의한다.
model = ConvNet().to(device) 
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
# epoch만큼 학습을 수행하고 cost를 갱신한다.
for epoch in range(epochs): 
    avg_cost = 0
    for data, target in train_loader:
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad() 
        hypothesis = model(data)
        cost = criterion(hypothesis, target) 
        cost.backward()
        optimizer.step() 
        avg_cost += cost / len(train_loader) 
    print('[Epoch: {:>4}]  cost = {:>.9}'.format(epoch + 1, avg_cost))

In [ ]:
# 검증모드로 바꾸고 모델의 성능을 검증한다.
model.eval()
with torch.no_grad(): 
    correct = 0
    total = 0

    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        out = model(data)
        preds = torch.max(out.data, 1)[1] 
        total += len(target) 
        correct += (preds==target).sum().item() 
        
    print('Test Accuracy: ', 100.*correct/total, '%')
     

## 첫 정규세션 들으시느라 고생 많으셨습니다.